# Project ML for portfolio management

The aim of this project is to explore gender differences in central bank communication / speeches using sentiment analysis, inspired by the paper "Leadership, Gender, and Discourse in Monetary Policy: Analyzing Speech Dynamics in the FOMC" (https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5002334).

We will explore gender communication styles in speeches delivered by central bank officials, focusing on whether there are observable differences between male and female central bank leaders.
More specifically, using sentiment analysis and NLP techniques, we will analyze whether male and female speakers differ in the topics they address (topic modeling) and the tone of their speeches (sentiment analysis).


The speeches would be scraped or downloaded from the BIS website, along with the speaker information. I would need to further define which speeches to use / focus on (based on major policy announcements for instance ?).

instructions : 1 : Sharp summary of our results. Should be clear if it works or not.
2 : dataset, can be short if reusing. Or longer if webscrapping  summary statistics, code, …
3 : why is it an important question, why does it matter ? (central bank data). What is your contribution, what do you bring ? what has been done and what changes with what I’m doing. What is new in what you’re doing.
Is the evaluation a backtest or something else, …
4 : results : explain what we’re doing and 
Summary of our test / empirical result. Can be positive or negative. 
5 : did it work or not, what can be improved / added ?
We can also / should !! read a paper and replicate it : google scholar 


1. Introduction
1. Dataset overview
1. Analytics and learning strategies
1. Empirical resuts: baseline and robustness 
1. Conclusion

if you need to add any package, no problem: add cells in your notebook with "pip install my_additional_package" so that I'm aware of what additional packages I need to run your notebook. 
if you use data that you scrapped online, just provide the code to programmatically scrape the data. More generally, I don't want to receive data.csv files. 
if you use .py files to tidy your project, just use a %%writefile magic in the notebook -- so that on my side, I can create the same .py files on the fly. I don't want to receive additional .py files.

In [ ]:
pip install selenium

### INTRODUCTION


The opening segment encompasses four essential elements:

- 1 Contextual Background: What is the larger setting of the study? What makes this area of inquiry compelling? What are the existing gaps or limitations within the current body of research? What are some unanswered yet noteworthy questions?

- 2 Project Contributions: What are the specific advancements made by this study, such as in data acquisition, algorithmic development, parameter adjustments, etc.?

- 3 Summary of the main empirical results: What is the main statistical statement? is it significant (e.g. statistically or economically)? 

- 4 Literature and Resource Citations: What are related academic papers? What are the github repositories, expert blogs, or software packages that used in this project? 

references : 
- Leadership, Gender, and Discourse in Monetary Policy: Analyzing Speech Dynamics in the FOMC (https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5002334)
- Information in Central Bank Sentiment: An Analysis of Fed and ECB Communication (https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4797935)

### DATASET OVERVIEW

In the dataset profile, one should consider:

- The origin and composition of data utilized in the study. If the dataset is original, then provide the source code to ensure reproducibility.
The dataset is composed of all central bankers' speeches avilable on the BIS website, coming from central banks institutions. The speeches date from 2009 up until 2024 (dates exactes?).
The data was requested through the BIS API and the speeches themselves were scraped from the BIS html pages.
/!\ the data collecting function takes a while to run as there are about 20000 speeches on the BIS website that are then filtered and then scrape corresponding speeches.
The get_central_bank_speeches_urls function requests all speeches individual urls and the load_central_banks_speeches requests all speeches metadata information and then scrapes the speeches content from the html pages if the filter of the institution correponds.


Assign gender

- The chronological accuracy of the data points, verifying that the dates reflect the actual availability of information.
Do a plot with nr of data points per year

- A detailed analysis of descriptive statistics, with an emphasis on discussing the importance of the chosen graphs or metrics.
Plot men vs women speakers

Central Bank Speeches: You need a corpus of speeches delivered by central bank officials, ideally annotated with speaker information (name, gender, role, country, etc.).
Metadata: To analyze gender differences, the dataset must include:
Gender of the speaker.
Date of the speech.
Context (policy announcements, conferences, etc.).

scope (e.g., BIS speeches from 2000 onwards)

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
from pathlib import Path

from tqdm import tqdm
# warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [3]:
# build functions
def get_central_bank_speeches_urls():
    '''
    '''
    url = "https://www.bis.org/api/document_lists/cbspeeches.json"
    reviews=[]
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        speeches = response.json()
        # Get the list of speeches ids ,
        for review in speeches['list']:
            reviews.append(review)
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
    
    return(reviews)

review_urls = get_central_bank_speeches_urls()
print(len(review_urls))

20093


In [ ]:
# NOT DONE AT ALL !!!!

 

def clean_directory_path(cache_dir, default_dir="data"):
    if cache_dir is None:
        cache_dir = Path(os.getcwd()) / default_dir
    if isinstance(cache_dir, str):
        cache_dir = Path(cache_dir)
    if not cache_dir.is_dir():
        os.makedirs(cache_dir)
    return cache_dir


logger = logging.getLogger(__name__)

def load_central_banks_speeches(add_url=True, cache_dir="data", force_reload=False, progress_bar=False, institutions_filter=[6]):
    """
    """
    filename = clean_directory_path(cache_dir) / "central_banks_speeches.parquet"
    # load from cache if it exists and not force_reload
    if (filename.exists()) and not force_reload:
        logger.info(f"loading from cache file: {filename}")
        speeches = pd.read_parquet(filename)
    else:
        logger.info("loading from external source")

        # get all speeches URLs
        urls = get_central_bank_speeches_urls()
        if progress_bar:
            review_urls_ = tqdm(urls)
        else:
            review_urls_ = urls


        # get speeches metadata + extract / scrape speech from html page
        all_speeches = []
        base_url_api = "https://www.bis.org/api/documents"
        base_url = "https://www.bis.org"

        # iterate over speeches URLs
        counter=0
        switch=None
        for link in review_urls_ : #tqdm(reviews):
            counter+=1
            speech_data = {}
            review_url = f'{base_url_api}{link}.json'
            speech_url = f'{base_url}{link}.htm'
            print(f"Processing speech: {review_url}")

            try:
                # Fetch speech page
                review_response = requests.get(review_url)
                review_metadata = review_response.json()
                speech_data.update(review_metadata)

                # Check if 'institutions' exists, skip processing if not
                    # only scrape speech if it's the right institution (filtering before scraping)  
                if 'institutions' in speech_data and any(inst in speech_data['institutions'] for inst in institutions_filter) : #speech_data['institutions'] == institutions_filter :
                    switch = counter
                    try:
                        # Scrape speech content
                        speech_response = requests.get(speech_url)
                        # Parse the HTML content
                        soup = BeautifulSoup(speech_response.content, 'html.parser')
                        # full speech is ocntained in the 'section' class
                        speech_content = soup.find('div', id='cmsContent')
                        speech_content_text = speech_content.get_text()
                        speech_data['speech_content'] = speech_content_text
                    
                        # append speech dict (content + metadata) to all_speeches
                        all_speeches.append(speech_data)

                    except Exception as e:
                        print(f"Failed to process speech: {speech_url}, error: {e}")
                        continue 

                else:
                    print(f"Skipping speech {review_url}: Institution not in filter or wrong institution")

            except Exception as e:
                    print(f"Failed to fetch data of review {review_url}, status code: {review_response.status_code}")
                    continue 
        
        # Convert to DataFrame
        speeches = pd.DataFrame(all_speeches)

        if add_url:
            speeches = speeches.assign(url=urls)
        
        # Save to cache
        logger.info(f"saving cache file {filename}")
        speeches.to_parquet(filename)
    return speeches

In [29]:
filename = Path('data')/ "central_banks_speeches.parquet"
print(filename)
filename.exists()

data\central_banks_speeches.parquet


False

In [27]:
def clean_directory_path(cache_dir, default_dir="data"):
    if cache_dir is None:
        cache_dir = Path(os.getcwd()) / default_dir
    if isinstance(cache_dir, str):
        cache_dir = Path(cache_dir)
    if not cache_dir.is_dir():
        os.makedirs(cache_dir)
    return cache_dir

clean_directory_path('data') / "central_banks_speeches.parquet"

WindowsPath('data/central_banks_speeches.parquet')

In [ ]:

# get speeches metadata + extract / scrape speech from html page
all_speeches = []
base_url_api = "https://www.bis.org/api/documents"
base_url = "https://www.bis.org"

counter = 0
switch = None
for link in review_urls : #tqdm(reviews):
     counter += 1
   #   link = '/review/r010105b'
     speech_data = {}
     review_url = f'{base_url_api}{link}.json'
     speech_url = f'{base_url}{link}.htm'
     print(f"Processing speech: {review_url}")
     try:
        # Fetch speech page
        review_response = requests.get(review_url)
        review_metadata = review_response.json()
        speech_data.update(review_metadata)

        # Check if 'institutions' exists, skip processing if not
             # only scrape speech if it's the right institution (filtering before scraping)  
        if 'institutions' in speech_data and speech_data['institutions'] == [6] :  #and 'publication_start_date' = 2024-12-06
            switch = counter

            try:
                # Scrape speech content
                speech_response = requests.get(speech_url)
                # Parse the HTML content
                soup = BeautifulSoup(speech_response.content, 'html.parser')
                # full speech is ocntained in the 'section' class
                speech_content = soup.find('div', id='cmsContent')
                speech_content_text = speech_content.get_text()
                speech_data['speech_content'] = speech_content_text

            except Exception as e:
                      print(f"Failed to process speech: {speech_url}, error: {e}")
                      continue 

              # append speech dict (content + metadata) to all_speeches
            all_speeches.append(speech_data)
     except Exception as e:
            print(f"Failed to fetch data of review {review_url}, status code: {review_response.status_code}")
            continue
     

# Save results
save_path = 'speeches.csv'
df = pd.DataFrame(all_speeches)
df.to_csv(save_path, index=False)
print(f"Scraped data saved to {save_path}")
df

# 30min 57

Processing speech: https://www.bis.org/api/documents/review/r010105b.json
Processing speech: https://www.bis.org/api/documents/review/r010115a.json
Processing speech: https://www.bis.org/api/documents/review/r010115b.json
Processing speech: https://www.bis.org/api/documents/review/r010115c.json
Processing speech: https://www.bis.org/api/documents/review/r010115d.json
Processing speech: https://www.bis.org/api/documents/review/r010117a.json
Processing speech: https://www.bis.org/api/documents/review/r010117b.json
Processing speech: https://www.bis.org/api/documents/review/r010117c.json
Processing speech: https://www.bis.org/api/documents/review/r010119a.json
Processing speech: https://www.bis.org/api/documents/review/r010119c.json
Processing speech: https://www.bis.org/api/documents/review/r010119d.json
Processing speech: https://www.bis.org/api/documents/review/r010124a.json
Processing speech: https://www.bis.org/api/documents/review/r010124b.json
Processing speech: https://www.bis.org

,id,path,menu_id,short_title,long_title,publication_start_date,language,pdf_file_size,pdf_pages,has_content,format,sources,authors,recurse_category,institutions,speech_content,topics,abstract
0,6657,/review/r090217b.pdf,256,Jean-Claude Trichet: Ten years of the euro - s...,"Speech by Mr Jean-Claude Trichet, President of...",2009-02-17T11:05:00.000+01:00,en,53.0,8.0,False,HTML_abstract,[Central bank],"[{'id': 241, 'author_type': 1, 'name': 'Jean-C...",[Speeches],[6],\n,NaN,NaN
1,6660,/review/r090217e.pdf,256,Lucas Papademos: Tackling the financial crisis...,"Speech by Mr Lucas Papademos, Vice President o...",2009-02-17T11:05:00.000+01:00,en,55.0,8.0,False,HTML_abstract,[Central bank],"[{'id': 253, 'author_type': 1, 'name': 'Lucas ...",[Speeches],[6],\n,NaN,NaN
2,6661,/review/r090218a.pdf,256,"Jean-Claude Trichet: Macroeconomic policies, i...","Address by Mr Jean-Claude Trichet, President o...",2009-02-18T11:05:00.000+01:00,en,29.0,3.0,False,HTML_abstract,[Central bank],"[{'id': 241, 'author_type': 1, 'name': 'Jean-C...",[Speeches],[6],\n,NaN,NaN
3,6665,/review/r090218e.pdf,256,Lorenzo Bini Smaghi: Regulation and supervisor...,"Speech by Mr Lorenzo Bini Smaghi, Member of th...",2009-02-18T11:05:00.000+01:00,en,37.0,6.0,False,HTML_abstract,[Central bank],"[{'id': 254, 'author_type': 1, 'name': 'Lorenz...",[Speeches],[6],\n,NaN,NaN
4,6666,/review/r090219a.pdf,256,Jean-Claude Trichet: Interview with China Finance,"Interview by Mr Jean-Claude Trichet, President...",2009-02-19T11:05:00.000+01:00,en,20.0,3.0,False,HTML_abstract,[Central bank],"[{'id': 241, 'author_type': 1, 'name': 'Jean-C...",[Speeches],[6],\n,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,37101,/review/r241126s.pdf,60,Christine Lagarde: Follow the money - channell...,"<p>Speech by Ms Christine Lagarde, President o...",2024-11-28,en,339.0,8.0,True,HTML_abstract,[Central bank],"[{'id': 3230, 'author_type': 1, 'name': 'Chris...",[Speeches],[6],"\n\n\n\n\nAt last year's conference, I spoke a...",NaN,NaN
1952,37111,/review/r241127f.pdf,60,Philip R Lane: Monetary policy under uncertainty,"<p><span>Keynote speech by Mr Philip R Lane, M...",2024-11-29,en,3849.0,30.0,True,HTML_abstract,[Central bank],"[{'id': 1411, 'author_type': 1, 'name': 'Phili...",[Speeches],[6],\n\nIntroduction\nMy aim today is to explain h...,[Inflation],NaN
1953,37112,/review/r241127g.pdf,60,Elizabeth McCaul: Objects in the rearview mirr...,<p><span>Keynote speech by Ms Elizabeth McCaul...,2024-11-29,en,16.0,6.0,True,HTML_abstract,[Central bank],"[{'id': 6407, 'author_type': 0, 'name': 'Eliza...",[Speeches],[6],\n\n\n\n\n\nIntroduction\nThank you very much ...,NaN,NaN
1954,37137,/review/r241202b.pdf,60,Philip R Lane: The 25th anniversary of the Eur...,"<p>Remarks by Mr Philip R Lane, Member of the ...",2024-12-02,en,263.0,5.0,True,HTML_abstract,[Central bank],"[{'id': 1411, 'author_type': 1, 'name': 'Phili...",[Speeches],[6],\n\nLet me start by congratulating the Euro50 ...,[Inflation],NaN


In [35]:
any(inst in speech_data['institutions'] for inst in [9, 6])

True

In [23]:
df_v1

,id,path,menu_id,short_title,long_title,publication_start_date,language,pdf_file_size,pdf_pages,has_content,format,sources,authors,recurse_category,institutions,speech_content,topics,abstract
0,6657,/review/r090217b.pdf,256,Jean-Claude Trichet: Ten years of the euro - s...,"Speech by Mr Jean-Claude Trichet, President of...",2009-02-17T11:05:00.000+01:00,en,53.0,8.0,False,HTML_abstract,[Central bank],"[{'id': 241, 'author_type': 1, 'name': 'Jean-C...",[Speeches],[6],\n,NaN,NaN
1,6660,/review/r090217e.pdf,256,Lucas Papademos: Tackling the financial crisis...,"Speech by Mr Lucas Papademos, Vice President o...",2009-02-17T11:05:00.000+01:00,en,55.0,8.0,False,HTML_abstract,[Central bank],"[{'id': 253, 'author_type': 1, 'name': 'Lucas ...",[Speeches],[6],\n,NaN,NaN
2,6661,/review/r090218a.pdf,256,"Jean-Claude Trichet: Macroeconomic policies, i...","Address by Mr Jean-Claude Trichet, President o...",2009-02-18T11:05:00.000+01:00,en,29.0,3.0,False,HTML_abstract,[Central bank],"[{'id': 241, 'author_type': 1, 'name': 'Jean-C...",[Speeches],[6],\n,NaN,NaN
3,6665,/review/r090218e.pdf,256,Lorenzo Bini Smaghi: Regulation and supervisor...,"Speech by Mr Lorenzo Bini Smaghi, Member of th...",2009-02-18T11:05:00.000+01:00,en,37.0,6.0,False,HTML_abstract,[Central bank],"[{'id': 254, 'author_type': 1, 'name': 'Lorenz...",[Speeches],[6],\n,NaN,NaN
4,6666,/review/r090219a.pdf,256,Jean-Claude Trichet: Interview with China Finance,"Interview by Mr Jean-Claude Trichet, President...",2009-02-19T11:05:00.000+01:00,en,20.0,3.0,False,HTML_abstract,[Central bank],"[{'id': 241, 'author_type': 1, 'name': 'Jean-C...",[Speeches],[6],\n,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,37101,/review/r241126s.pdf,60,Christine Lagarde: Follow the money - channell...,"<p>Speech by Ms Christine Lagarde, President o...",2024-11-28,en,339.0,8.0,True,HTML_abstract,[Central bank],"[{'id': 3230, 'author_type': 1, 'name': 'Chris...",[Speeches],[6],"\n\n\n\n\nAt last year's conference, I spoke a...",NaN,NaN
1952,37111,/review/r241127f.pdf,60,Philip R Lane: Monetary policy under uncertainty,"<p><span>Keynote speech by Mr Philip R Lane, M...",2024-11-29,en,3849.0,30.0,True,HTML_abstract,[Central bank],"[{'id': 1411, 'author_type': 1, 'name': 'Phili...",[Speeches],[6],\n\nIntroduction\nMy aim today is to explain h...,[Inflation],NaN
1953,37112,/review/r241127g.pdf,60,Elizabeth McCaul: Objects in the rearview mirr...,<p><span>Keynote speech by Ms Elizabeth McCaul...,2024-11-29,en,16.0,6.0,True,HTML_abstract,[Central bank],"[{'id': 6407, 'author_type': 0, 'name': 'Eliza...",[Speeches],[6],\n\n\n\n\n\nIntroduction\nThank you very much ...,NaN,NaN
1954,37137,/review/r241202b.pdf,60,Philip R Lane: The 25th anniversary of the Eur...,"<p>Remarks by Mr Philip R Lane, Member of the ...",2024-12-02,en,263.0,5.0,True,HTML_abstract,[Central bank],"[{'id': 1411, 'author_type': 1, 'name': 'Phili...",[Speeches],[6],\n\nLet me start by congratulating the Euro50 ...,[Inflation],NaN


219 : 1min7s cours

In [6]:
67*10000/60/60  # --> 3h pour 20000 lignes

186.11111111111111

In [ ]:

def load_loughran_mcdonald_dictionary(cache_dir="data", force_reload=False):
    """
    Software Repository for Accounting and Finance by Bill McDonald
    https://sraf.nd.edu/loughranmcdonald-master-dictionary/
    """
    filename = (
        clean_directory_path(cache_dir)
        / "Loughran-McDonald_MasterDictionary_1993-2021.csv"
    )
    if (filename.exists()) & (~force_reload):
        logger.info(f"logging from cache file: {filename}")
    else:
        logger.info("loading from external source")
        id = "17CmUZM9hGUdGYjCXcjQLyybjTrcjrhik"
        url = f"https://docs.google.com/uc?export=download&confirm=t&id={id}"        
        subprocess.run(f"wget -O '{filename}' '{url}'", shell=True, capture_output=True)
    return pd.read_csv(filename)

In [ ]:

def load_10X_summaries(cache_dir="data", force_reload=False):
    """
    Software Repository for Accounting and Finance by Bill McDonald
    https://sraf.nd.edu/sec-edgar-data/
    """
    filename = (
        clean_directory_path(cache_dir)
        / "Loughran-McDonald_10X_Summaries_1993-2021.csv"
    )
    if (filename.is_file()) & (~force_reload):
        logger.info(f"logging from cache directory: {filename}")
    else:
        logger.info("loading from external source")
        id = "1CUzLRwQSZ4aUTfPB9EkRtZ48gPwbCOHA"
        url = f"https://docs.google.com/uc?export=download&confirm=t&id={id}"
        subprocess.run(f"wget -O '{filename}' '{url}'", shell=True, capture_output=True)

    df = pd.read_csv(filename).assign(
        date=lambda x: pd.to_datetime(x.FILING_DATE, format="%Y%m%d")
    )
    return df



### ANALYTICS AND LEARNING STRATEGY

The analytics and machine learning methodologies section accounts for:

- A detailed explanation of the foundational algorithm.

- A description of the data partitioning strategy for training, validation and test.

- An overview of the parameter selection and optimization process.

### EMPIRICAL RESULTS : BASELINE AND ROBUSTNESS

To effectively convey the empirical findings, separate the baseline results from the additional robustness tests. Within the primary empirical outcomes portion, include:

- Key statistical evaluations (for instance, if presenting a backtest – provide a pnl graph alongside the Sharpe ratio).

- Insights into what primarily influences the results, such as specific characteristics or assets that significantly impact performance.

The robustness of empirical tests section should detail:

- Evaluation of the stability of the principal finding against variations in hyperparameters or algorithmic modifications.

### CONCLUSION

Finally, the conclusive synthesis should recapitulate the primary findings, consider external elements that may influence the results, and hint at potential directions for further investigative work.